In [2]:
import sys

# cd to ../src
sys.path.append("../src")

In [3]:
from model.load_model import load_model_for_inference
import random
import tempfile
import json


import logging
import os


import torch
import torch.nn as nn


from transformers import (
    PreTrainedTokenizerBase,
)
from transformers.modeling_utils import unwrap_model
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES

logging.basicConfig(level=logging.INFO)

In [5]:
model_baseline, tokenizer_baseline = load_model_for_inference(
    weights_path="/gaueko1/hizkuntza-ereduak/LLaMA/lm/huggingface/7B/",
    quantization=4,
    lora_weights_name_or_path="/ikerlariak/osainz006/models/collie/CoLLIE-7b_lora4_baseline",
)

INFO:root:Loading model from /gaueko1/hizkuntza-ereduak/LLaMA/lm/huggingface/7B/
INFO:root:We will load the model using the following device map: None
INFO:root:Bits and Bytes config: {
    "load_in_8bit": false,
    "load_in_4bit": true,
    "llm_int8_threshold": 6.0,
    "llm_int8_skip_modules": null,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "bnb_4bit_compute_dtype": "float16"
}



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /ikerlariak/igarcia945/envs/pytorch2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so


/ikerlariak/igarcia945/envs/pytorch2/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('pmix-server'), PosixPath('2964042')}
  warn(msg)
/ikerlariak/igarcia945/envs/pytorch2/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/java/java16')}
  warn(msg)
/ikerlariak/igarcia945/envs/pytorch2/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/soft_ixa_linux')}
  warn(msg)
/ikerlariak/igarcia945/envs/pytorch2/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('40465'), PosixPath('//127.0.0.1'), P

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /ikerlariak/igarcia945/envs/pytorch2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:root:Model dtype: torch.float16
INFO:root:Total model memory footprint: 3796.381696 MB
INFO:root:Loading pretrained LORA weights from /ikerlariak/osainz006/models/collie/CoLLIE-7b_lora4_baseline


In [6]:
model, tokenizer = load_model_for_inference(
    weights_path="/gaueko1/hizkuntza-ereduak/LLaMA/lm/huggingface/7B/",
    quantization=4,
    lora_weights_name_or_path="/ikerlariak/osainz006/models/collie/CoLLIE-7b_lora4",
)

INFO:root:Loading model from /gaueko1/hizkuntza-ereduak/LLaMA/lm/huggingface/7B/
INFO:root:We will load the model using the following device map: None
INFO:root:Bits and Bytes config: {
    "load_in_8bit": false,
    "load_in_4bit": true,
    "llm_int8_threshold": 6.0,
    "llm_int8_skip_modules": null,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "bnb_4bit_compute_dtype": "float16"
}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:root:Model dtype: torch.float16
INFO:root:Total model memory footprint: 3796.381696 MB
INFO:root:Loading pretrained LORA weights from /ikerlariak/osainz006/models/collie/CoLLIE-7b_lora4


In [16]:
prompt = (
    '# The following lines describe the task definition\n@dataclass\nclass Person(Entity):\n    """Persons: first,'
    ' middle and last names of people, animals and fictional\n            characters\n    aliases"""\n\n    span:'
    ' str\n\n\n@dataclass\nclass Organization(Entity):\n    """Organizations: companies (press agencies, studios,'
    " banks, stock\n               markets, manufacturers, cooperatives)\n    subdivisions of companies (newsrooms)\n "
    "   brands\n    political movements (political parties, terrorist\n              organisations)\n    government"
    " bodies (ministries, councils, courts, political unions\n               of countries (e.g. the {\\\\it U.N.}))\n "
    "   publications (magazines, newspapers, journals)\n    musical companies (bands, choirs, opera companies,"
    " orchestras\n    public organisations (schools, universities, charities\n    other collections of people (sports"
    " clubs, sports\n              teams, associations, theaters companies,\n              religious orders, youth"
    ' organisations\n    """\n\n    span: str\n\n\n@dataclass\nclass Location(Entity):\n    """Locations: roads'
    " (streets, motorways)\n    trajectories\n    regions (villages, towns, cities, provinces, countries,"
    " continents,\n             dioceses, parishes)\n    structures (bridges, ports, dams)\n    natural locations"
    " (mountains, mountain ranges, woods,\n                       rivers, wells, fields, valleys, gardens,\n          "
    "             nature reserves, allotments, beaches,\n                       national parks)\n    public places"
    " (squares, opera houses, museums, schools,\n                   markets, airports, stations, swimming pools,\n    "
    "               hospitals, sports facilities, youth centers,\n                   parks, town halls, theaters,"
    " cinemas, galleries,\n                   camping grounds, NASA launch pads, club\n                   houses,"
    " universities, libraries, churches,\n                   medical centers, parking lots, playgrounds,\n            "
    "       cemeteries)\n    commercial places (chemists, pubs, restaurants, depots,\n                       hostels,"
    " hotels, industrial parks,\n                       nightclubs, music venues)\n    assorted buildings (houses,"
    " monasteries, creches, mills,\n                        army barracks, castles, retirement\n                      "
    "  homes, towers, halls, rooms, vicarages,\n                        courtyards)\n    abstract ``places'' (e.g."
    ' {\\\\it the free world})\n    """\n\n    span: str\n\n\n# This is the text to analyze\ntext = "In any event ,'
    ' this question is not presently included among the requests for topical and urgent debate on Thursday ."\n\n# The'
    " annotation instances that take place in the text above are listed here\nresult = ["
)
print(prompt)

# The following lines describe the task definition
@dataclass
class Person(Entity):
    """Persons: first, middle and last names of people, animals and fictional
            characters
    aliases"""

    span: str


@dataclass
class Organization(Entity):
    """Organizations: companies (press agencies, studios, banks, stock
               markets, manufacturers, cooperatives)
    subdivisions of companies (newsrooms)
    brands
    political movements (political parties, terrorist
              organisations)
    government bodies (ministries, councils, courts, political unions
               of countries (e.g. the {\\it U.N.}))
    publications (magazines, newspapers, journals)
    musical companies (bands, choirs, opera companies, orchestras
    public organisations (schools, universities, charities
    other collections of people (sports clubs, sports
              teams, associations, theaters companies,
              religious orders, youth organisations
    """

    span: str




In [85]:
prompt_guideline = (
    '# The following lines describe the task definition\n@dataclass\nclass TwoWheelVehicle(Entity):\n    '
    '"""Vehicles that have two wheels, for example, motorbikes and bycicles"""\n\n    span:'
    ' str\n\n\n@dataclass\nclass FourWheelVehicle(Entity):\n'
    '"""Vehicles that have four wheels, for example, cars and trucks"""\n\n    span:str\n\n\n'
    '# This is the text to analyze\ntext = '
    '"The car overtake the motorbike very fast"'
    ' \n\n# The'
    " annotation instances that take place in the text above are listed here\nresult ="
)
print(prompt_guideline)

# The following lines describe the task definition
@dataclass
class TwoWheelVehicle(Entity):
    """Vehicles that have two wheels, for example, motorbikes and bycicles"""

    span: str


@dataclass
class FourWheelVehicle(Entity):
"""Vehicles that have four wheels, for example, cars and trucks"""

    span:str


# This is the text to analyze
text = "The car overtake the motorbike very fast" 

# The annotation instances that take place in the text above are listed here
result =


In [103]:
prompt_guideline = (
    '# The following lines describe the task definition\n@dataclass\nclass Vehicle2(Entity):\n    '
    '"""Vehicles that have two wheels, for example, bicycles"""\n\n    span:'
    ' str\n\n\n@dataclass\nclass Vehicle4(Entity):\n'
    '"""Vehicles that have four wheels, for example, automobile"""\n\n    span:str\n\n\n'
    '# This is the text to analyze\ntext = '
    '"The car overtake the motorbike very fast"'
    ' \n\n# The'
    " annotation instances that take place in the text above are listed here\nresult ="
)
print(prompt_guideline)

# The following lines describe the task definition
@dataclass
class Vehicle2(Entity):
    """Vehicles that have two wheels, for example, bicycles"""

    span: str


@dataclass
class Vehicle4(Entity):
"""Vehicles that have four wheels, for example, automobile"""

    span:str


# This is the text to analyze
text = "The car overtake the motorbike very fast" 

# The annotation instances that take place in the text above are listed here
result =


In [104]:
prompt_clear = (
    '# The following lines describe the task definition\n@dataclass\nclass TwoWheelVehicle(Entity):\n    '
    '""""""\n\n    span:'
    ' str\n\n\n@dataclass\nclass FourWheelVehicle(Entity):\n'
    '""""""\n\n    span:str\n\n\n'
    '# This is the text to analyze\ntext = '
    '"The car overtake the motorbike very fast"'
    ' \n\n# The'
    " annotation instances that take place in the text above are listed here\nresult ="
)
print(prompt_clear)

# The following lines describe the task definition
@dataclass
class TwoWheelVehicle(Entity):
    """"""

    span: str


@dataclass
class FourWheelVehicle(Entity):
""""""

    span:str


# This is the text to analyze
text = "The car overtake the motorbike very fast" 

# The annotation instances that take place in the text above are listed here
result =


In [105]:
model_input = tokenizer(prompt_guideline, add_special_tokens=True, return_tensors="pt")
model_input["input_ids"]=model_input["input_ids"][:,:-1]
model_input

{'input_ids': tensor([[    1,   396,   450,  1494,  3454,  8453,   278,  3414,  5023,    13,
         29992,  1272,  1990,    13,  1990,  8980, 29882,  2512, 29906, 29898,
          6691,  1125,    13,  1678,  9995, 29963, 14797,  4027,   393,   505,
          1023, 21266,  1379, 29892,   363,  1342, 29892,   289,  4245,  7799,
         15945, 29908,    13,    13,  1678, 10638, 29901,   851,    13,    13,
            13, 29992,  1272,  1990,    13,  1990,  8980, 29882,  2512, 29946,
         29898,  6691,  1125,    13, 15945, 29908, 29963, 14797,  4027,   393,
           505,  3023, 21266,  1379, 29892,   363,  1342, 29892,  3345,  5249,
         15945, 29908,    13,    13,  1678, 10638, 29901,   710,    13,    13,
            13, 29937,   910,   338,   278,  1426,   304, 27599,    13,   726,
           353,   376,  1576,  1559,   975, 19730,   278, 10992,  5365,   446,
          1407,  5172, 29908, 29871,    13,    13, 29937,   450, 17195,  8871,
           393,  2125,  2058,   297,  

In [106]:
#CoLLIE
model_ouput = model.generate(input_ids=model_input.input_ids.to(model.device), max_new_tokens=128,do_sample=False,min_new_tokens=0,num_beams=1)
print(tokenizer.batch_decode(model_ouput)[0])

<s> # The following lines describe the task definition
@dataclass
class Vehicle2(Entity):
    """Vehicles that have two wheels, for example, bicycles"""

    span: str


@dataclass
class Vehicle4(Entity):
"""Vehicles that have four wheels, for example, automobile"""

    span:str


# This is the text to analyze
text = "The car overtake the motorbike very fast" 

# The annotation instances that take place in the text above are listed here
result = [
    Vehicle4(span="car"),
    Vehicle2(span="motorbike"),
]
</s>


In [107]:
#model_input = tokenizer(prompt_clear, add_special_tokens=True, return_tensors="pt")
#model_input["input_ids"]=model_input["input_ids"][:,:-1]
#model_input

model_ouput = model_baseline.generate(input_ids=model_input.input_ids.to(model_baseline.device), max_new_tokens=128,do_sample=False,min_new_tokens=0,num_beams=1)
print(tokenizer.batch_decode(model_ouput)[0])

<s> # The following lines describe the task definition
@dataclass
class Vehicle2(Entity):
    """Vehicles that have two wheels, for example, bicycles"""

    span: str


@dataclass
class Vehicle4(Entity):
"""Vehicles that have four wheels, for example, automobile"""

    span:str


# This is the text to analyze
text = "The car overtake the motorbike very fast" 

# The annotation instances that take place in the text above are listed here
result = [
    Vehicle2(span="car"),
    Vehicle2(span="motorbike"),
]
</s>
